#Mount Azure Data Lake containers for project


In [0]:
def mount_adls(storage_account_name, container_name):
    # Get secrets from Key Vault
    client_id = dbutils.secrets.get(scope="formula1-scope", key="formula1-app-client-id")
    tenant_id = dbutils.secrets.get(scope="formula1-scope", key="formula1-app-tenant-id")
    client_secret = dbutils.secrets.get(scope="formula1-scope", key= "formula1-app-client-secret")
    # Set Spark configuration 
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    # check if container already exist
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    # mount the storage account
    dbutils.fs.mount(
      source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = configs)
    #display(dbutils.fs.mounts())
    

####Mount Raw container

In [0]:
mount_adls("formula1dl712", "raw")


#### Mount Processed container

In [0]:
mount_adls("formula1dl712", "processed")

#### Mount Presentation container

In [0]:
mount_adls("formula1dl712", "presentation")

#### Mount demo1 container

In [0]:
mount_adls("formula1dl712", "demo1")

#### Mount streaming-demo container

In [0]:
mount_adls("formula1dl712", "streaming-demo")

In [0]:
display(dbutils.fs.mounts())

#### Mount demo container

In [0]:
display(spark.read.csv("/mnt/formula1dl712/demo"))

#### Unmount demo container

In [0]:
dbutils.fs.unmount("/mnt/formula1dl712/demo")